In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

In [ ]:
## Config
DATASET_DIR = "dataset"
MODEL_DIR = "model"
os.makedirs(MODEL_DIR, exist_ok=True)

IMG_SIZE = 180
BATCH_SIZE = 16
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
## Transforms
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [ ]:
## Dataset and Loader
train_data = datasets.ImageFolder(DATASET_DIR, transform=transform)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
class_names = train_data.classes

In [ ]:
## Save class names
with open(os.path.join(MODEL_DIR, "class_names.txt"), "w") as f:
    f.write("\n".join(class_names))

In [ ]:
## Model
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(class_names))
model = model.to(DEVICE)

In [ ]:
## Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
## Training Loop
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{EPOCHS}: Loss={running_loss:.4f}, Accuracy={acc:.2f}%")

✅ Epoch 1/10: Loss=94.7497, Accuracy=77.62%
✅ Epoch 2/10: Loss=43.4557, Accuracy=89.66%
✅ Epoch 3/10: Loss=29.6786, Accuracy=93.26%
✅ Epoch 4/10: Loss=25.4574, Accuracy=93.42%
✅ Epoch 5/10: Loss=18.8664, Accuracy=95.11%
✅ Epoch 6/10: Loss=17.8976, Accuracy=95.68%
✅ Epoch 7/10: Loss=10.9166, Accuracy=97.89%
✅ Epoch 8/10: Loss=15.7553, Accuracy=95.94%
✅ Epoch 9/10: Loss=17.3483, Accuracy=95.88%
✅ Epoch 10/10: Loss=15.6635, Accuracy=95.99%


In [ ]:
## Save model
torch.save(model.state_dict(), os.path.join(MODEL_DIR, "model.pt"))
print("Training complete Model saved")  

Training complete. Model saved.
